In [ ]:
from glob import glob

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from ultralytics import YOLO

In [ ]:
def polygon_to_mask(polygon, width, height):
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1)
    mask = np.array(img)
    return mask

In [ ]:
model = YOLO('../runs/segment/train/weights/best.pt')

In [ ]:
plt.figure(figsize=(14, 10))
for i, image in enumerate(glob('../images/test_images/*')):
    results = model(image, verbose=False)

    plt.subplot(3, 3, i+1)
    plt.imshow(cv2.cvtColor(results[0].orig_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    for res in results:
        object_class = int(res[0].boxes.cls.item())
        for xy in res[0].masks.xy:
            mask = polygon_to_mask(xy, res[0].orig_shape[1], res[0].orig_shape[0])
            mask = np.repeat(np.expand_dims(mask, -1), 4, -1) * 255
            mask[:, :, object_class%3] = 0
            plt.imshow(mask, alpha=0.3)
            plt.title(res[0].names[object_class])